In [32]:
include("./mnist_cnn.jl")

┌ Info: Train data
│   typeof(x_train_) = Array{Float32, 4}
│   size(x_train_) = (28, 28, 1, 60000)
│   typeof(y_train_) = Matrix{Float32}
│   size(y_train_) = (10, 60000)
└ @ Main /Users/zhenchen/Julia_MNIST_VAE/data.jl:13
┌ Info: Test data
│   typeof(x_test_) = Array{Float32, 4}
│   size(x_test_) = (28, 28, 1, 10000)
│   typeof(y_test_) = Matrix{Float32}
│   size(y_test_) = (10, 10000)
└ @ Main /Users/zhenchen/Julia_MNIST_VAE/data.jl:24


┌ Info: Model
│   model_ = Chain(Conv((3, 3), 1 => 32, relu, pad=1), MaxPool((2, 2)), Conv((3, 3), 32 => 32, relu, pad=1), MaxPool((2, 2)), Conv((3, 3), 32 => 32, relu, pad=1), flatten, Dropout(0.5), Dense(1568 => 128, elu), Dropout(0.5), Dense(128 => 10, elu), softmax)
└ @ Main /Users/zhenchen/Julia_MNIST_VAE/mnist_cnn.jl:64


In [33]:
model_ = modelF()

Chain(
  Conv((3, 3), 1 => 32, relu, pad=1),   # 320 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 32 => 32, relu, pad=1),  # 9_248 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 32 => 32, relu, pad=1),  # 9_248 parameters
  Flux.flatten,
  Dropout(0.5),
  Dense(1568 => 128, elu),              # 200_832 parameters
  Dropout(0.5),
  Dense(128 => 10, elu),                # 1_290 parameters
  NNlib.softmax,
)                   # Total: 10 arrays, 220_938 parameters, 864.516 KiB.

In [31]:
x_train_[1:28, 1:28, 1, 1:2]

28×28×2 Array{Float32, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0       …  0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.215686  0.533333  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0       …  0.67451   0.992157  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.886275  0.992157  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.992157  0.992157  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.992157  0.831373  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.992157  0.529412  0.0  0.0  0.0
 ⋮                        ⋮         ⋱                      ⋮         
 0.0  0.0  0.0  0.0  0.0  0.101961     0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.65098   …  0.0       0.0       0.0  0.0  0.0
 0.0  0.0  0.0 